In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
num_channels = 3
max_sequence_length = 1000  # Set maximum sequence length

def dataPrep(folderPath):
    # Get a list of all .npy files in the folder
    file_list = [file for file in os.listdir(folderPath) if file.endswith('.npy')]

    #print(len(file_list))
    # Initialize lists to store padded data
    padded_data_list = []

    # Process each .npy file
    for file_name in file_list:
        # Load data from .npy file
        data = np.load(os.path.join(folder_pathTest, file_name))

        # Truncate or pad sequences to max_sequence_length
        truncated_data = [seq[:max_sequence_length] for seq in data]
        padded_data = pad_sequences(truncated_data, maxlen=max_sequence_length, padding='pre', dtype='float32')

        padded_data_list.append(padded_data)
    padded_data_list.append(padded_data_list[0]) 
    # Concatenate data from all files
    padded_data_all = np.stack(padded_data_list, axis=0) #1136

    # Swap axes to match model input shape (None, 1000, 3)
    padded_data_all = np.swapaxes(padded_data_all, 1, 2)
    return padded_data_all

In [11]:
folder_pathTest="C:\\Users\\91629\\Desktop\\MF_test_data"
test_data=dataPrep(folder_pathTest)

In [12]:
label1=pd.read_pickle("C:\\Users\\91629\\Downloads\\test_data_mf2_label.pkl")

In [13]:
test_data.shape

(1137, 1000, 3)

In [14]:
label1.shape

(1137, 677)

In [21]:
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(max_sequence_length, num_channels)),
    MaxPooling1D(pool_size=2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification, adjust output units for multiclass
])

model.add(Flatten())
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(test_data, label1, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
29/29 [==============================] - 4s 73ms/step - loss: 0.1306 - accuracy: 0.9603 - val_loss: 0.0834 - val_accuracy: 0.9847
Epoch 2/10
29/29 [==============================] - 2s 60ms/step - loss: 0.0656 - accuracy: 0.9883 - val_loss: 0.0797 - val_accuracy: 0.9847
Epoch 3/10
29/29 [==============================] - 2s 59ms/step - loss: 0.0641 - accuracy: 0.9883 - val_loss: 0.0797 - val_accuracy: 0.9847
Epoch 4/10
29/29 [==============================] - 2s 64ms/step - loss: 0.0643 - accuracy: 0.9883 - val_loss: 0.0795 - val_accuracy: 0.9847
Epoch 5/10
29/29 [==============================] - 2s 63ms/step - loss: 0.0639 - accuracy: 0.9883 - val_loss: 0.0799 - val_accuracy: 0.9847
Epoch 6/10
29/29 [==============================] - 2s 63ms/step - loss: 0.0639 - accuracy: 0.9883 - val_loss: 0.0802 - val_accuracy: 0.9847
Epoch 7/10
29/29 [==============================] - 2s 67ms/step - loss: 0.0637 - accuracy: 0.9883 - val_loss: 0.0801 - val_accuracy: 0.9847
Epoch 8/10
29

In [24]:
test_metrics=model.evaluate(test_data,label1, verbose=0)

In [25]:
print("loss during training of train_data:",test_metrics[0])

loss during training of train_data: 0.06666439771652222


In [26]:
print("accuracy during training of test_data:",test_metrics[1])

accuracy during training of test_data: 0.987538754940033


In [27]:
predicted_data=model.predict(test_data)

36/36 [==============================] - 1s 14ms/step


In [28]:
actual_data=label1.values

In [29]:
predicted_data

array([[0.0105283 ],
       [0.01063498],
       [0.01053161],
       ...,
       [0.01512757],
       [0.01236262],
       [0.0105283 ]], dtype=float32)

In [30]:
actual_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)